In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news-classification/WELFake_Dataset.csv


In [55]:
df = pd.read_csv('/kaggle/input/fake-news-classification/WELFake_Dataset.csv')  # importing the dataset

In [56]:
# dropping the null values of the dataset
df = df.dropna()

In [57]:
df.drop(columns = 'Unnamed: 0',inplace = True)

In [58]:
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [59]:
x = df.drop(columns = ['text','label'],axis= 1)

In [60]:
y = df['label']

In [61]:
x

,title
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,SATAN 2: Russia unvelis an image of its terrif...
5,About Time! Christian Group Sues Amazon and SP...
...,...
72129,Russians steal research on Trump in hack of U....
72130,WATCH: Giuliani Demands That Democrats Apolog...
72131,Migrants Refuse To Leave Train At Refugee Camp...
72132,Trump tussle gives unpopular Mexican leader mu...


In [62]:
y.value_counts()

label
1    36509
0    35028
Name: count, dtype: int64

In [63]:
x.shape

(71537, 1)

In [64]:
y.shape

(71537,)

In [65]:
# importing the dependencies
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Bidirectional

In [67]:
messages = x.copy()

In [68]:
messages.reset_index(inplace = True)

In [69]:
import nltk   # a library used in text preprocessing
import re
from nltk.corpus import stopwords

In [70]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [71]:
# performing stemming and removing the stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])   # [^a-zA-Z] is used to remove full stop,punctuations and other symbols other than alphabets.
    review = review.lower()   # lowercasing all the text
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

# Applying Tokenizer on the preprocessed data

In [84]:
from keras.preprocessing.text import Tokenizer

In [85]:
tokenizer = Tokenizer()

In [86]:
docs = ''

for i in corpus:
    docs = docs + i

In [88]:
tokenizer.fit_on_texts([docs])     # tokenizing all the text data

In [90]:
len(tokenizer.word_index)

70930

In [125]:
tokenizer.texts_to_sequences([corpus[1]])

[[2460, 8, 366, 212, 4, 2327, 1869, 332, 50, 266, 14, 44, 671, 39]]

In [99]:
x = []

for i in corpus:
    x.append(tokenizer.texts_to_sequences([i])[0])

In [101]:
max_len = max([len(seq) for seq in x])

In [102]:
max_len

47

In [103]:
# embedding all the sentences
sent_length = 50
embedded_docs = pad_sequences(x,padding = 'post',maxlen = sent_length)
print(embedded_docs)

[[  95 1108  315 ...    0    0    0]
 [2460    8  366 ...    0    0    0]
 [6857 6573  345 ...    0    0    0]
 ...
 [ 313  243  165 ...    0    0    0]
 [   1 7060   70 ...    0    0    0]
 [2226 2052  277 ...    0    0    0]]


In [104]:
x_input = np.array(embedded_docs)
y_input = np.array(y)

In [107]:
y_input

array([1, 1, 0, ..., 0, 0, 1])

In [127]:
# Creating the model
from tensorflow.keras.layers import Dropout
embed_vec_ftr = 40
model = Sequential()
model.add(Embedding(70930,embed_vec_ftr,input_length = sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 40)            2837200   
                                                                 
 dropout (Dropout)           (None, 50, 40)            0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 2,950,201
Trainable params: 2,950,201
Non-trainable params: 0
____________________________________________

In [129]:
x_input.shape,y_input.shape

((71537, 50), (71537,))

In [130]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x_input,y_input,test_size = 0.33,random_state = 2)

In [131]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_loss', patience=3)

In [132]:
model.fit(x_train,y_train,validation_data = (x_test,y_test),epochs = 5,batch_size = 32)

Epoch 1/5
1498/1498 [==============================] - 32s 19ms/step - loss: 0.2908 - accuracy: 0.8717 - val_loss: 0.2239 - val_accuracy: 0.9085
Epoch 2/5
1498/1498 [==============================] - 16s 11ms/step - loss: 0.1908 - accuracy: 0.9257 - val_loss: 0.2194 - val_accuracy: 0.9115
Epoch 3/5
1498/1498 [==============================] - 16s 11ms/step - loss: 0.1461 - accuracy: 0.9425 - val_loss: 0.2255 - val_accuracy: 0.9149
Epoch 4/5
1498/1498 [==============================] - 16s 10ms/step - loss: 0.1211 - accuracy: 0.9532 - val_loss: 0.2473 - val_accuracy: 0.9116
Epoch 5/5
1498/1498 [==============================] - 16s 10ms/step - loss: 0.1012 - accuracy: 0.9614 - val_loss: 0.2766 - val_accuracy: 0.9077


In [133]:
y_pred = model.predict(x_test)

738/738 [==============================] - 4s 3ms/step


In [134]:
y_pred = np.where(y_pred>=0.5,1,0)

In [135]:
y_pred

array([[0],
       [0],
       [1],
       ...,
       [1],
       [0],
       [0]])

In [136]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)  # accuracy of the model

0.9077007793968146

In [137]:
confusion_matrix(y_test,y_pred)

array([[10373,  1186],
       [  993, 11056]])

In [138]:
# predictive model
def fake_news(text):
    
    review = re.sub('[^a-zA-Z]',' ', text)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    
    input_seq = tokenizer.texts_to_sequences([review])[0]
    
    pad_seq = pad_sequences([input_seq],padding = 'post',maxlen = sent_length)
    
    output = (model.predict(pad_seq))[0]
    
    return np.where(output>0.5,1,0)[0]
    

In [143]:
fake_news("Asaram served notice to vacate over 17,000 sq mt land")

1/1 [==============================] - 0s 22ms/step


1